# Frank's Tutorial of the Reweighing Technique
##### Author: Guanzhong Chen
##### Date: 04/17/2020
Reference: F. Kamiran and T. Calders,  "Data Preprocessing Techniques for Classification without Discrimination," Knowledge and Information Systems, 2012.

## 1. Introduction to Bias in Machine Learning

This tutorial is meant to introduce one of the techniques in AI Fairness 360 (AIF360) package called "Reweighing Technique." The AIF360 toolkit is an open-source library the helps machine learning researchers and the whole community detect and mitigate bias in machine learning models.

To give a brief background introduction of bias in machine learning, we look at a very simple dataset. This dataset classifies people described by a set of attributes as good or bad credit risks. File used is `german.data` consisting of 1000 instances and 20 features. In this case we focus on the supervised machine learning problem with a binary target of the credit risks being either "good" or "bad." A machine learning model will learn and generalize the pattern from a training dataset and make predictions on a test dataset based on what it has learned. However, here is a problem. The training dataset may not be representative of the true population of people of all age groups. For example, in the training dataset, people with ages more than 25 are much more likely to receive a good credit risk due to the source of the dataset or some other reasons. However, the true distribution might be otherwise. This will generate bias and be unfavorable for people with ages less than 25. In this case, "age" will be our protected attribute, and it separates the instances into two groups: more than 25 and less than 25.

Before further investigation, let's first import the german dataset, set the protected attribute, set the threshold of separation, set the training and testing dataset, and drop other sensitive attributes.

In [1]:
# Load all necessary packages
import sys
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

Matplotlib Error, comment out matplotlib.use('TkAgg')


In [2]:
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           
    privileged_classes=[lambda x: x >= 25],      
    features_to_drop=['personal_status', 'sex'] 
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

## 2. Preprocessing Techniques to Mitigate Bias

There are many solutions to mitigate bias in machine learning. In this tutorial, we focus on those that are proceeded before training. They are known as the "preprocessing" techniques.

We list three of them here that are commonly used. We give a brief introduction to what they are and what their pros and cons are to help readers to choose the one that is most suitable for them.

1. **Suppresion**: First, we identify the attributes that are most correlated with the sensitive attribute S. Then, we just remove S and these most correlated attribute. 
    * **Pros**: the algorithm itself is straightforward to understand and easy to implement. 
    * **Cons**: Sometimes we can't get rid of the sensitive attributes easily by just removing them. Some of them may be critical for companies' bussiness analysis, and some of them may be important for the classification process.

2. **Dataset Massaging**: We change the labels of some objects in the dataset. Selections of the labels to change will base on a ranker that is related to Naive Bayes classifier.
    * **Pros**: It help mitigate bias even if the sentitive attributes are not allowed to be removed. It is also relatively easy to understand.
    * **Cons**: It is, in a sense, rather intrusive as it changes the labels of the instances.

2. **Reweighing**: We give training instances weights according to the law of statistical independence.
    * **Pros**: It is calculation-friendly. We can implement this algorithm using frequency count. Compared to dataset massaging, it also helps mitigate bias without changing the labels.
    * **Cons**: More difficult to implement compared to the two techniques mentioned above.

## 3. Elaboration on Reweighing

To have an idea of how this technique works, we first introduce some notations. We assume the sensitive attribute and the target variable are binary. Specifically, we denote the sensitive attribute as $S$ with two values $\{b,w\}$. We denote the target class as $T$ with two values $\{+, -\}$. The classifier we use is denoted as $C$, and the random unlabeled data subject is denoted as $X$. Also the training dataset is denoted as $D$.

If the dataset $D$ is unbiased, $S$ and $T$ are statistically independent. This means the expected probability can be calculated as following:  

$$
P_{exp}(S=b \wedge T=+) = \frac{|\{X \in D | X(S) = b\}|}{|D|} * \frac{|\{X \in D | X(T) = +\}|}{|D|}
$$

In reality, the observed probability is calculated as following:  

$$
P_{obs}(S=b \wedge T=+) = \frac{|\{X \in D | X(S) = b \wedge X(T) = +\}|}{|D|}
$$

We notice that $P_{exp}$ and $P_{obs}$ are usually different. We assign the weight for each instance to be W that is calculated as following:

$$
W(X) = \frac{P_{exp}(S=b \wedge T=+)}{P_{obs}(S=b \wedge T=+)}
$$

In [3]:
print(sum(dataset_orig_train.protected_attributes))

[587.]


In [4]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

In [5]:
print(RW.w_p_fav) # both are 1.0
print(RW.w_p_unfav)
print(RW.w_up_fav)
print(RW.w_up_unfav)

0.9622950819672131
1.100625
1.2555555555555555
0.678


In [6]:
print(dir(dataset_transf_train))
print(dataset_transf_train.privileged_protected_attributes)

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_de_dummy_code_df', '_parse_feature_names', 'align_datasets', 'convert_to_dataframe', 'copy', 'export_dataset', 'favorable_label', 'feature_names', 'features', 'ignore_fields', 'import_dataset', 'instance_names', 'instance_weights', 'label_names', 'labels', 'metadata', 'privileged_protected_attributes', 'protected_attribute_names', 'protected_attributes', 'scores', 'split', 'subset', 'temporarily_ignore', 'unfavorable_label', 'unprivileged_protected_attributes', 'validate_dataset']
[array([1.])]


In [7]:
print(sum(dataset_transf_train.protected_attributes == True))

[587]


In [8]:
print(sum(dataset_transf_train.protected_attributes == False))

[113]


In [9]:
print(sum(dataset_transf_train.convert_to_dataframe()[0]["age"] == 1.0))

587


In [10]:
print(sum(dataset_transf_train.convert_to_dataframe()[0]["credit"] == 1.0))

490


In [11]:
print(sum(dataset_transf_train.convert_to_dataframe()[0]["credit"] == 2.0))

210


In [12]:
dataset_transf_train.convert_to_dataframe()[0]["age"]

993    1.0
859    1.0
298    1.0
553    1.0
672    1.0
      ... 
509    1.0
340    0.0
221    0.0
928    1.0
146    1.0
Name: age, Length: 700, dtype: float64

In [13]:
df = dataset_transf_train.convert_to_dataframe()[0]
df[(df['age']==1.0)&(df['credit']==1.0)].shape

(427, 58)